To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + support us if you can!
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

In [1]:
# https://www.kaggle.com/code/hotchpotch/llm-detect-pip 
!pip install -q -U accelerate --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/

Kaggle credentials set.
Kaggle credentials successfully validated.


* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.
* [**NEW**] We make Gemma 6 trillion tokens **2.5x faster**! See our [Gemma notebook](https://colab.research.google.com/drive/10NbwlsRChbma1v55m8LAPYG15uQv6HLo?usp=sharing)

In [1]:
import torch
import random
import numpy as np
import pandas as pd
import gc
import time
import os

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    IS_SUBMISSION = True
else:
    # exit to not waste gpu
    IS_SUBMISSION = False

#https://github.com/Lightning-AI/lit-gpt/issues/327
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("Sorry - GPU required!")

import logging
logging.getLogger('transformers').setLevel(logging.ERROR)

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [2]:
model_name = '/kaggle/input/mistral-7b-it-v02'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# Embedding seed exmaple
seed_str = """prompt,text,cluster
Rewrite the essay as though it were a news article,"Hi Charles, \n\nI hope this email finds you well. I wanted to let you know that I have reviewed your recent presentation, and I must say that your forte in public speaking really shines through. Your ability to engage the audience and deliver the message effectively is impressive. Keep up the great work! \n\nBest regards,\n[Your Name]",1
Rewrite the story as a romance,"Patricia, a passionate baker, carefully poured the rich, golden syrup over her freshly baked pancakes. The warm, sweet aroma filled her cozy kitchen as she eagerly anticipated the first bite. The syrup, made from the finest maple trees in her family's farm, added a delightful touch to her breakfast. With each mouthful, Patricia savored the perfect balance of sweetness and smoothness that the syrup brought to her pancakes. It was a simple pleasure that made her mornings brighter and her breakfasts unforgettable.",2
Rewrite the essay as a noir detective story,"In the fields of golden wheat,\nA laborer named Pete,\nWith calloused hands and weary feet,\nToiled under the summer's heat.\n\nFrom dawn till dusk, he'd sow and reap,\nHis sweat and tears, a promise to keep,\nFor his family's future, he'd always leap,\nWith each harvest, their dreams would steep.\n\nDonna, his wife, would wait at home,\nWith love and patience, she'd never roam,\nTogether they'd build a life of their own,\nWith hard work, their seeds would be sown.\n\nIn the end, their labor would prevail,\nA testament to their unwavering tale,\nFor the laborer's spirit shall never fail,\nIn the fields, their hopes would never pale.",4
Rewrite the essay as a comedy,"Mildred woke up feeling sick. She dragged herself to work, but her boss noticed her pale face. Worried, he sent her home. Mildred spent the day in bed, sipping warm tea and watching movies. Her best friend, Sarah, came over with soup and a get-well card. Mildred smiled, grateful for the care. The next day, she felt better and thanked Sarah for being there when she was sick.",5
Rewrite the essay as a mystery/thriller,"In a twist of circumstance, Dorothy found herself stranded in a small town called Serendipity. With no phone signal and a dead car battery, she was left with no choice but to explore her surroundings. As she wandered the charming streets, Dorothy stumbled upon a hidden gem of a café. The warm atmosphere and friendly locals quickly made her forget about her predicament. Serendipity turned out to be the perfect place to embrace the unexpected and find joy in the most unforeseen circumstances.",6
Rewrite the opening sentence to be more engaging and impactful.,Hey Arlene! Just wanted to let you know that I booked our tickets for the trip out west. Can't wait to explore the beautiful landscapes and experience the adventure together. See you soon!,7
Rewrite the essay as a dystopian piece,"Ladies and gentlemen, thank you for being here today. Life is a series of fleeting moments, and sometimes we are lucky enough to catch a glimpse of something truly extraordinary. Just like Howard, who recently had the chance to witness a breathtaking sunset over the majestic mountains. In that brief moment, he felt a sense of awe and wonder that reminded him of the beauty that surrounds us every day. Let us remember to cherish these glimpses, for they have the power to inspire, uplift, and remind us of the magic that exists in the world.",8
Rewrite the essay without using adjectives,"Title: Compliance Report on Safety Measures in Accordance with Company Policies\n\nDate: [Date]\n\nObjective: To assess the adherence to safety protocols in accordance with company policies at [Company Name].\n\nSummary: This report provides an overview of the compliance status regarding safety measures at [Company Name]. The assessment was conducted by a team of safety inspectors led by George, the Safety Compliance Officer. The inspection covered various areas, including emergency exits, fire extinguisher placement, and employee training. Overall, the company demonstrated a high level of accordance with safety policies, with only minor recommendations for improvement.\n\nKey Findings:\n1. Emergency Exits: All emergency exits were clearly marked",9
"Rewrite the story, but replace realism with absurdity .","Dear Dr. Smith,\n\nI am writing to inquire about your recent research on genetics. I am particularly interested in your findings regarding the role of genetic mutations in cancer development. Could you please provide me with more information on this topic? I am eager to learn more about the potential implications for treatment and prevention strategies.\n\nThank you for your time and consideration.\n\nSincerely,\nKimberly",11
Rewrite the story as if you were describing this experience to a child,"Hey Brenda! Just wanted to let you know that I found a great deal on a workout apparatus online. It's perfect for your home gym. Check it out here: [link]. Let me know if you're interested, I can help you order it.",12
Rewrite the story as a futuristic sci-fi,"Dear Carol,\n\nI hope this letter finds you well. I wanted to let you know that I have discovered my forte in painting. It brings me immense joy and fulfillment. I would love to show you some of my recent works and hear your thoughts. Perhaps we can plan a visit to the art gallery together soon. Looking forward to catching up with you.\n\nWarm regards,\n[Your Name]",13
Rewrite the sentence in a more concise and engaging way.,"Hi Janice, \n\nJust wanted to let you know that there will be a temporary closure of the highway tomorrow from 9 am to 12 pm for maintenance work. Please plan your commute accordingly. \n\nThanks, \n[Your Name]",14
Rewrite the story from the first person perspective,"In the small town of Oakville, the majority of residents were excited about the upcoming summer festival. Darren, a local musician, was thrilled to be performing on the main stage. The festival committee had worked hard to ensure there would be something for everyone, from food trucks to live performances. Darren's band, The Oakville Groove, had been practicing tirelessly to entertain the majority of festival-goers. As the event drew closer, anticipation grew, and the majority of Oakville's population eagerly awaited the festivities, ready to celebrate their community spirit.",15
Rewrite the story from the perspective of the character villain,"Yesterday, at the local market, Cordelia accidentally dropped her wallet. Unbeknownst to her, it was picked up by a mischievous teenager named Max. Max quickly stole the cash and discarded the wallet. Luckily, a kind stranger found it and returned it to Cordelia, who was relieved to have her belongings back. The incident served as a reminder to Cordelia to be more cautious with her belongings in crowded places.",17
Rewrite the story with a positive or happy twist .,"Dear Maria, \n\nI hope this letter finds you well. I wanted to let you know that the basement renovation is complete. It looks amazing! The new flooring and fresh paint have really transformed the space. I can't wait for you to see it. Let's plan a get-together soon so you can check it out. \n\nBest regards, \n[Your Name]",18
"Rewrite the essay as a cheerful, uplifting, inspiring post about friendship","Memo\n\nTo: All Staff\nFrom: [Your Name]\nDate: [Today's Date]\n\nSubject: Encouragement Challenge\n\nDear Team,\n\nI hope this memo finds you well. As we navigate these challenging times, it's important to support and uplift one another. I am excited to announce the ""Encouragement Challenge""! \n\nStarting today, let's make a conscious effort to offer words of encouragement to at least one colleague each day. Whether it's a simple compliment or a note of appreciation, let's spread positivity throughout our workplace. Remember, a little encouragement can go a long way in boosting morale and fostering a supportive environment.\n\nLet's make a difference together!\n\nBest regards,\n\n[Your Name]",20
Rewrite the essay with a horror twist,"Hey Esther! Did you know that aardvarks are fascinating creatures? They have long snouts and eat ants and termites. I saw a video of one digging into an anthill, it was so cool! Just thought you might find it interesting. Have a great day!",21
Rewrite the essay so the story is told through tweets.,"In the bustling city of Crestville, Joan was a determined entrepreneur who had built a successful bakery called Sweet Delights. However, her dreams were shattered when a large corporation, Silver Spoon Confections, decided to take over the local bakery scene. Joan refused to give up and devised a plan to regain control. With the help of loyal customers and her talented team, she organized a city-wide bake-off. The event showcased the unique flavors and creativity of Sweet Delights, capturing the hearts of the community. In the end, Joan's bakery triumphed, proving that passion and perseverance can conquer any takeover.",22
Rewrite the essay as a Shakespearean play,"In the small town of Willowbrook, Patsy discovered a hidden book in her attic. As she read the ancient pages, she learned about a mystical necklace that granted eternal life. Determined to find it, Patsy embarked on a journey through treacherous forests and dark caves. Finally, she reached the secret chamber where the necklace lay. With trembling hands, she placed it around her neck, feeling the weight of eternity settle upon her shoulders.",27
Rewrite the story to have a religious angle,"Cameron, a young and ambitious entrepreneur, launched his own line of deodorants called ""Fresh Essence."" With a unique blend of natural ingredients, his deodorants quickly gained popularity. However, Cameron faced a setback when a rival company released a similar product with a catchy name. Determined to stand out, he decided to partner with a local charity and donate a portion of his profits to a cause close to his heart. The community rallied behind Cameron's mission, and ""Fresh Essence"" became the go-to deodorant, not just for its effectiveness, but also for its positive impact on society.",28
Rewrite the essay as if it were a video game.,"During the thunderstorm, Dan and his friends huddled together, laughing and telling stories, creating memories that would last a lifetime.",30
"Rewrite the same story, but where that character is not human.","In the dense jungles of Africa, Arlene, a renowned primatologist, embarked on a thrilling expedition to study gorillas. With unwavering determination, she observed their intricate social behaviors and documented their impressive strength. Arlene's groundbreaking research shed light on the intelligence and vulnerability of these majestic creatures, inspiring conservation efforts worldwide.",31
Rewrite the essay as a poem in verse/rhyme,"Dear Michele, \nJust wanted to let you know that we're meeting at The Tipsy Tavern tonight at 8 pm. Hope to see you there for some drinks and good times! \nCheers, \n[Your Name]",36
Rewrite the text in a more informal tone.,"Catherine, a skilled hacker, was hired to do some hackwork for a high-profile client. With her expertise, she swiftly infiltrated the target's system, extracting valuable information without leaving a trace. The client was impressed by Catherine's efficiency and discretion, ensuring her reputation as the go-to hacker for any hackwork needs.",39
Rewrite the story as a murder mystery,"Hey Thomas, just a quick reminder to send me the file for the presentation before 5 pm today. We need to review it and make any necessary changes before tomorrow's meeting. Thanks!",46
Rewrite the essay as if it took place in the Harry Potter universe,"David strolled down the bustling boulevard, mesmerized by the vibrant lights and the lively atmosphere. It was a place where dreams came alive, and he couldn't help but feel alive too.",51
Rewrite the story in the style of an action movie,"Subject: Biopsy Results\nDear Team, \nDr. Smith's biopsy results for Jeffrey came back negative. No further action required. \nBest, \n[Your Name]",52
Rewrite the essay with time travel elements,"Rebecca eagerly awaited the results of her research study, hoping it would validate her hypothesis. After months of hard work, the data finally confirmed her theory, bringing a sense of accomplishment and validation to her efforts. She was excited to share her findings with her colleagues and contribute to the scientific community's knowledge on the subject.",59"""

with open('seed_final.csv', 'w') as f:
    f.write(seed_str)

In [3]:
# seed
import pandas as pd
from pathlib import Path

DATA_PATH = Path("./")

seed_df = pd.read_csv(DATA_PATH / "seed_final.csv")
seed_df.head()

,rewrite_prompt,original_text,rewritten_text,reasoning
0,Rewrite the essay as though it were a news art...,"Hi Charles, \n\nI hope this email finds you we...",**Local Speaker Impresses with Eloquent Delive...,The rewritten essay has been rewritten to read...
1,Rewrite the story as a romance,"Patricia, a passionate baker, carefully poured...","In the quaint coziness of her kitchen, Patrici...",The rewritten text aligns with the rewrite req...
2,Rewrite the essay as a noir detective story,"In the fields of golden wheat,\nA laborer name...","The sun dipped low, casting long shadows acros...","Pete, akin to a detective, navigates the chall..."
3,Rewrite the essay as a comedy,Mildred woke up feeling sick. She dragged hers...,Mildred woke up feeling the pits of her stomac...,Mildred's misadventures with sickness turn int...
4,Rewrite the essay as a mystery/thriller,"In a twist of circumstance, Dorothy found hers...","The rain poured down violently, soaking Doroth...",The serene setting of Serendipity transforms i...


In [17]:
import torch
import torch.nn.functional as F
import random

device = 'cuda'

def get_cot_prompt(row):
    user_prompt='Original Text: """{}"""\n\nGive me the rewrite prompt that should be used to change the above original text into the following text.\n"""{}"""'
    model_prompt='The rewrite prompt is: """{prompt}""". Let\'s think step by step. {reasoning}'

    return [
        {"role": "user", "content": user_prompt.format(row['original_text'], row['rewritten_text']) },
        {"role": "assistant", "content": model_prompt.format(reasoning=row['reasoning'], prompt=row['rewrite_prompt']) },
    ]

def get_prompt(original, rewritten):
    rows = [row for _, row in seed_df.iterrows()]
    random.shuffle(rows)
    rows = random.sample(rows, k=8)
    cot = []
    for row in rows:
        cot += get_cot_prompt(row)
    user_prompt='Original Text: """{}"""\n\nGive me the rewrite prompt that should be used to change the above original text into the following text.\n"""{}"""'
    cot += [
        {"role": "user", "content":user_prompt.format(original, rewritten)},
        # {"role": "assistant", "content": 'The rewrite prompt is: """Improve this text by'},
    ]
    return cot

prompt 1: [{'role': 'user', 'content': 'Original Text: """Hey Esther! Did you know that aardvarks are fascinating creatures? They have long snouts and eat ants and termites. I saw a video of one digging into an anthill, it was so cool! Just thought you might find it interesting. Have a great day!"""\n\nGive me the rewrite prompt that should be used to change the above original text into the following text.\n"""Hey Esther, did you know that aardvarks are fascinating creatures? They have long snouts and eat ants and termites. I saw a video of one digging into an anthill last night, and it was... well, let\'s just say that I\'m not sleeping with the lights off again.\n\nThe video started with a standard shot of a desert landscape. However, as the camera zoomed in, the seemingly mundane"""'}, {'role': 'assistant', 'content': 'The rewrite prompt is: """Rewrite the essay with a horror twist""". Let\'s think step by step. The rewrite transforms the lighthearted topic into a horror-themed narr

# Predict experiment

In [18]:
import torch
from sentence_transformers import SentenceTransformer

t5base_model = None

# Function to calculate sharpened cosine similarity
def sharpened_cosine_similarity(vec1, vec2, exponent=3):
    cosine_similarity = torch.nn.functional.cosine_similarity(vec1, vec2, dim=0)
    return cosine_similarity ** exponent

#provides similarity scores of a test_phrase against an array of phrases
def compare_phrases(test_phrase, phrases):
    global t5base_model
    t5base_model = SentenceTransformer('sentence-t5-base')
    if torch.cuda.is_available():
        t5base_model = t5base_model.to(torch.device("cuda"))
    model = t5base_model

    scores = []
    test_embedding = model.encode(test_phrase, convert_to_tensor=True, show_progress_bar=False)

    for phrase in phrases:
        compare_embedding = model.encode(phrase, convert_to_tensor=True, show_progress_bar=False)
        score = sharpened_cosine_similarity(test_embedding, compare_embedding).item()
        scores.append(score)

    return scores

In [19]:
import pandas as pd

if IS_SUBMISSION:
    df = pd.read_csv(DATA_PATH / "nbroad-v2.csv")
    df = df.iloc[:10]
else:
    df = pd.read_csv(DATA_PATH / "nbroad-v2.csv")

In [20]:
import tqdm
import numpy as np
from scipy import stats

device = 'cuda'

df_test = df.iloc[:100]

print('Predict')
predicts = []
for i, row in tqdm.tqdm(df_test.iterrows(), total=df_test.shape[0]):
    original_text = row['original_text']
    rewritten_text = row['rewritten_text']

    candidates = []
    for i in range(1):
        prompt = get_prompt(original_text, rewritten_text)

        encodeds = tokenizer.apply_chat_template(prompt, return_tensors="pt")

        model_inputs = encodeds.to(device)

        generated_ids = model.generate(model_inputs, max_new_tokens=50, do_sample=False, pad_token_id=tokenizer.eos_token_id)
        decoded = tokenizer.batch_decode([
            generated_ids[i][len(model_inputs[i]):]
            for i in range(len(model_inputs))
        ])

        result = decoded[0]
        if 'Let\'s think step by step' in result:
            result = result.split('Let\'s think step by step')[0]
        result = result.replace('The rewrite prompt is:', '').replace('"""', '').strip()
        base_prompt = f'Improve this by {result}'
        candidates.append(base_prompt)

    result = candidates[0]
    predicts.append(result)

df_test['predict'] = predicts

if IS_SUBMISSION:
    df_submission = df_test[['id', 'predict']]
    df_submission['rewrite_prompt'] = df_submission['predict']
    df_submission.to_csv(DATA_PATH / 'submission.csv', index=False, columns=['id', 'rewrite_prompt'])

print('Scoring')
# calc score
scores = []
for i, row in tqdm.tqdm(df_test.iterrows(), total=df_test.shape[0]):
    scores.append(compare_phrases(row['rewritten_text'], [row['predict'], ])[0])

df_test['scores'] = scores
df_test.to_csv(DATA_PATH / 'test_100.csv', index=False)

print('\nTest score stats: ', stats.describe(np.array(scores)))
print('\nMean SCS score: ', np.mean(np.array(scores)))

df_test.head()

Predict


100%|██████████| 100/100 [55:59<00:00, 33.60s/it]
<ipython-input-20-04e3752ecdb3>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predict'] = predicts


Scoring


100%|██████████| 100/100 [01:59<00:00,  1.19s/it]


Test score stats:  DescribeResult(nobs=100, minmax=(0.3126964569091797, 0.6335738301277161), mean=0.4732864218950272, variance=0.004061855402932755, skewness=0.38095941718630516, kurtosis=0.03495500987180389)

Mean SCS score:  0.4732864218950272



<ipython-input-20-04e3752ecdb3>:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['scores'] = scores


,id,original_text,rewrite_prompt,rewritten_text,predict,scores
0,lZGdiueMer,"`` Well, there are healthier ways to tell me y...",Rewrite the story where the writer asks the re...,"Well, there are healthier ways to tell me you ...",Improve this by The rewrite prompt that could ...,0.563337
1,DfTJVFKrUk,Rory ran his shaky fingers through his wife's ...,Rewrite the essay as a dramatic play,## The Final Curtain\n\n[FADE IN]\n\n**Setting...,Improve this by The rewrite prompt that could ...,0.435212
2,LmJvKranXK,As I made my way on foot across town to the Po...,Rewrite the story with all the themes and sett...,As I made my way through the Tatooine desert o...,Improve this by The rewrite prompt that could ...,0.528898
3,PpnqXQAdGH,`` Hello. We come in peace.'' \n \n The first ...,Rewrite the essay if the advanced aliens didn'...,`` Hello. We come in peace.''\n\nThe first enc...,Improve this by The given text is already in t...,0.398539
4,qOeXTfqgAM,"`` Karen, what the helllllll izzz...'' says my...",Rewrite the story as a court room drama starri...,The courtroom erupted in an uproar as District...,Improve this by The rewrite prompt that could ...,0.472389
